# ***Visualization***

#### **At the given Market place, they care a lot about production per hour (pph). The provided set of data is used to visualise for the given metrics.**

1. Make a table showing the pph (items produced per hour worked), breaking out each location within each market, monthly.

2. Some execs like to look just at Product A production, some prefer to include Product B, so please make a parameter in a drop-down that lets them choose to look at Product A only in this table or include Product B too.

3. In the tooltip on this table, show what percent of that month and year’s total production (across all locations in the company) that this location accounted for.

4. In the tooltip on this table, show the year-to-date for pph for each store.

5. In the tooltip on this table, show the year-over-year for that month

6. In the tooltip on this table, show the year-over-year comparison of the year-to-date pph.

7. Make it possible to collapse/roll up the stores to the markets and calculate the pph numbers (and tooltip) correctly still.

8. Tell us something about this data in an interesting visual way in a separate viz.


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import warnings
from matplotlib import pyplot as pl
import plotly.express as px
warnings.simplefilter(action='ignore')

df=pd.read_csv("/content/data.csv")
df.head()

,work_date,work_week,store_id,Site Name,metric,quantity,city
0,1/2/2015,12/28/2014,1232,Store A,Product A Produced,29,LA
1,1/2/2015,12/28/2014,1232,Store A,Hours Worked,98,LA
2,1/9/2015,1/4/2015,6453,Store B,Product B Produced,34,San Diego
3,1/9/2015,1/4/2015,6453,Store B,Product A Produced,77,San Diego
4,1/5/2015,1/4/2015,6453,Store B,Hours Worked,113,San Diego


In [ ]:
for i in df['Site Name'].unique():
  print("The respective store id for the site ",df[df['Site Name']==i]['store_id'].unique()," is ",i)

The respective store id for the site  [1232]  is  Store A
The respective store id for the site  [6453]  is  Store B
The respective store id for the site  [1234]  is  Store C
The respective store id for the site  [5678]  is  Store D
The respective store id for the site  [2356]  is  Store E
The respective store id for the site  [1290]  is  Store F
The respective store id for the site  [7859]  is  Store G

The sites  ['Store A'] present in the city  LA
The sites  ['Store B' 'Store C' 'Store E'] present in the city  San Diego
The sites  ['Store D'] present in the city  San Francisco
The sites  ['Store F'] present in the city  San Mateo
The sites  ['Store G'] present in the city  Irvine


In [ ]:
# Sites in defined city
for i in df['city'].unique():
  print("The sites ",df[df['city']==i]['Site Name'].unique(),"present in the city ",i)

In [ ]:
df['metric'].unique()

array(['Product A Produced', 'Hours Worked', 'Product B Produced'],
      dtype=object)

import pandas as pd
import numpy as np


df=pd.read_csv("data.csv")
df['date'] = pd.to_datetime(df['work_date'], format='%m/%d/%Y')
df['work_date'] = df['date'].dt.strftime('%d/%m/%Y')
df['date'] = pd.to_datetime(df['work_week'], format='%m/%d/%Y')
df['work_week'] = df['date'].dt.strftime('%d/%m/%Y')
df=df.loc[:, df.columns != 'date']
year,month=[],[]
for i in df["work_date"]:
    idx=i.split("/")
    year.append(idx[2])
    month.append(idx[1])
df['year']=year
df['month']=month


## **Preprocessing**

# New section

In [ ]:
# run once
df['date'] = pd.to_datetime(df['work_date'], format='%m/%d/%Y')
df['work_date'] = df['date'].dt.strftime('%d/%m/%Y')
df['date'] = pd.to_datetime(df['work_week'], format='%m/%d/%Y')
df['work_week'] = df['date'].dt.strftime('%d/%m/%Y')
df=df.loc[:, df.columns != 'date']
year,month,date=[],[],[]
for i in df["work_date"]:
    idx=i.split("/")
    date.append(idx[0])
    month.append(idx[1])
    year.append(idx[2])
df['date']=date  
df['month']=month
df['year']=year

df

,work_date,work_week,store_id,Site Name,metric,quantity,city,date,month,year
0,02/01/2015,28/12/2014,1232,Store A,Product A Produced,29,LA,02,01,2015
1,02/01/2015,28/12/2014,1232,Store A,Hours Worked,98,LA,02,01,2015
2,09/01/2015,04/01/2015,6453,Store B,Product B Produced,34,San Diego,09,01,2015
3,09/01/2015,04/01/2015,6453,Store B,Product A Produced,77,San Diego,09,01,2015
4,05/01/2015,04/01/2015,6453,Store B,Hours Worked,113,San Diego,05,01,2015
...,...,...,...,...,...,...,...,...,...,...
6346,04/12/2016,04/12/2016,7859,Store G,Hours Worked,105,Irvine,04,12,2016
6347,30/12/2016,25/12/2016,1290,Store F,Hours Worked,83,San Mateo,30,12,2016
6348,27/12/2016,25/12/2016,1290,Store F,Hours Worked,103,San Mateo,27,12,2016
6349,31/12/2016,25/12/2016,1290,Store F,Hours Worked,115,San Mateo,31,12,2016


In [ ]:
df['store_id'].unique()

array([1232, 6453, 1234, 5678, 2356, 1290, 7859])

In [ ]:
df['metric'].unique()

array(['Product A Produced', 'Hours Worked', 'Product B Produced'],
      dtype=object)

1. Make a table showing the pph (items produced per hour worked), breaking out each location within each market, monthly.

In [ ]:
dfq1=pd.DataFrame(columns=['new_city','sum_worked_hours','sum_quantity','pph','ytd_pph'])
for i in df['city'].unique():
    dff=df[df['city']==i] 
    tot_hours=sum(dff[dff['metric']=='Hours Worked']['quantity'])
    tot_quantity=sum(dff[dff['metric']=='Product A Produced']['quantity'])
    dfq1=dfq1.append({'new_city':i,'sum_worked_hours':tot_hours,'sum_quantity':tot_quantity,'pph':float(tot_quantity/tot_hours)},ignore_index=True)
display(dfq1)

In [ ]:
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(dfq1['new_city'], dfq1['pph'], color ='blue',
        width = 0.4)
plt.xlabel("City",fontsize=15)
plt.ylabel("PPH (production per hour)",fontsize=15)
plt.title("PPH in each City",fontsize=20)
plt.show()

2. Some execs like to look just at Product A production, some prefer to include Product B, so please make a parameter in a drop-down that lets them choose to look at Product A only in this table or include Product B too.

In [ ]:
data=df[df['metric']=='Product '+input('Enter A/B:')+' Produced']
display(data.loc[:, data.columns != 'metric'])

Enter A/B:A


,work_date,work_week,store_id,Site Name,quantity,city,year,month
0,02/01/2015,28/12/2014,1232,Store A,29,LA,2015,01
3,09/01/2015,04/01/2015,6453,Store B,77,San Diego,2015,01
11,08/01/2015,04/01/2015,1234,Store C,28,San Diego,2015,01
12,07/01/2015,04/01/2015,1234,Store C,72,San Diego,2015,01
13,06/01/2015,04/01/2015,1234,Store C,54,San Diego,2015,01
...,...,...,...,...,...,...,...,...
6331,21/11/2016,20/11/2016,7859,Store G,29,Irvine,2016,11
6339,28/11/2016,27/11/2016,7859,Store G,51,Irvine,2016,11
6340,01/12/2016,27/11/2016,7859,Store G,65,Irvine,2016,12
6341,05/12/2016,04/12/2016,7859,Store G,29,Irvine,2016,12


3. In the tooltip on this table, show what percent of that month and year’s total production (across all locations in the company) that this location accounted for.

In [ ]:
list_df=[]
month_dict={'01':'jan','02':'feb','03':'mar','04':'apr','05':'may','06':'jun','07':'jul','08':'aug','09':'sep','10':'oct','11':'nov','12':'dec'}
for years in df['year'].unique():
  print(years)
  df_2015=pd.DataFrame(columns=['city','jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'])
  df_2015['city']=df['city'].unique()
  for month in df['month'].unique():
    total_city=sum(df[df['year']==years][df['metric']!='Hours Worked'][df['month']==month]['quantity'])
    calculated={}  
    for city in df['city'].unique():
      per_city_month=sum(df[df['year']==years][df['metric']!='Hours Worked'][df['month']==month][df['city']==city]['quantity'])
      calculated[city]=round((per_city_month/total_city)*100,2)
    df_2015[month_dict[month]]=calculated.values()
  display(df_2015)
  list_df.append(df_2015)

2015


,city,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
0,LA,6.73,10.12,8.82,8.08,8.88,13.96,7.96,5.32,13.58,12.56,12.87,6.19
1,San Diego,58.18,50.72,41.32,43.14,44.39,34.62,34.54,46.85,42.32,42.12,46.26,42.28
2,San Francisco,11.75,19.34,19.17,21.72,21.15,14.88,9.89,14.28,16.35,19.60,13.12,23.97
3,San Mateo,7.87,10.48,14.63,8.85,11.65,17.39,17.52,17.64,17.76,16.50,18.78,15.60
4,Irvine,15.47,9.34,16.06,18.22,13.94,19.15,30.09,15.91,9.98,9.23,8.97,11.96


2016


,city,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
0,LA,10.16,9.77,7.53,12.31,14.49,15.32,8.34,8.90,10.19,12.34,13.21,13.09
1,San Diego,51.65,53.27,52.52,48.17,50.53,40.68,39.37,23.63,39.10,36.36,30.14,20.69
2,San Francisco,13.32,15.15,14.03,9.53,2.94,14.37,17.38,25.73,18.59,16.24,11.61,9.50
3,San Mateo,12.14,10.66,13.12,15.49,19.58,14.46,20.46,27.37,21.13,22.51,33.30,45.77
4,Irvine,12.73,11.16,12.80,14.50,12.46,15.17,14.46,14.37,10.99,12.55,11.74,10.94


In [ ]:
month=['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']
num=int(input("Enter the month number: "))
wide_df = px.data.medals_wide()

fig = px.bar(df, x=list_df[1]["city"], y=df_2015[month[num-1]], title=(month[num-1]).upper())
fig.show()

Enter the month number: 1


4. In the tooltip on this table, show the year-to-date for pph for each store.

In [ ]:
df["pph"]=dfq1["pph"]
df.head()
df['work_date'] = pd.to_datetime(df['work_date'])
dfq1['ytd_pph'] = df.groupby(['store_id', df['work_date'].dt.year])['pph'].cumsum()
dfq1
df["ytd_pph"]=dfq1["ytd_pph"]
df
fig = px.line(df, x='work_date', y='pph', color='store_id', 
              hover_data=['ytd_pph'], title='PPH by Store over Time')
fig.show()

5. In the tooltip on this table, show the year-over-year for that month

In [ ]:
df['yoy_pph'] = df.groupby(['Site Name', df['work_date'].dt.year, df['work_date'].dt.month])['ytd_pph'].cumsum()*100/12
fig = px.line(df, x='work_date', y='pph', color='store_id', 
              hover_data=['yoy_pph'], title='PPH by Store over Time')
fig.show()
df

,work_date,work_week,store_id,Site Name,metric,quantity,city,pph,ytd_pph,yoy_pph
0,2015-01-02,12/28/2014,1232,Store A,Product A Produced,29,LA,0.164979,0.164979,1.374827
1,2015-01-02,12/28/2014,1232,Store A,Hours Worked,98,LA,0.334623,0.499602,5.538177
2,2015-01-09,1/4/2015,6453,Store B,Product B Produced,34,San Diego,0.368916,0.368916,3.074297
3,2015-01-09,1/4/2015,6453,Store B,Product A Produced,77,San Diego,0.302627,0.671543,8.670487
4,2015-01-05,1/4/2015,6453,Store B,Hours Worked,113,San Diego,0.297208,0.968751,16.743408
...,...,...,...,...,...,...,...,...,...,...
6346,2016-12-04,12/4/2016,7859,Store G,Hours Worked,105,Irvine,NaN,NaN,NaN
6347,2016-12-30,12/25/2016,1290,Store F,Hours Worked,83,San Mateo,NaN,NaN,NaN
6348,2016-12-27,12/25/2016,1290,Store F,Hours Worked,103,San Mateo,NaN,NaN,NaN
6349,2016-12-31,12/25/2016,1290,Store F,Hours Worked,115,San Mateo,NaN,NaN,NaN


6. In the tooltip on this table, show the year-over-year comparison of the year-to-date pph.

7. Make it possible to collapse/roll up the stores to the markets and calculate the pph numbers (and tooltip) correctly still.

In [ ]:
dfq7=pd.DataFrame(columns=['Site Name','sum_worked_hours','sum_quantity','pph'])
for i in df['Site Name'].unique():
    dff=df[df['Site Name']==i] 
    tot_hours=sum(dff[dff['metric']=='Hours Worked']['quantity'])
    tot_quantity=sum(dff[dff['metric']=='Product A Produced']['quantity'])
    dfq7=dfq7.append({'Site Name':i,'sum_worked_hours':tot_hours,'sum_quantity':tot_quantity,'pph':float(tot_quantity/tot_hours)},ignore_index=True)
display(dfq7)

,Site Name,sum_worked_hours,sum_quantity,pph
0,Store A,46224,7626,0.164979
1,Store B,44584,13413,0.300848
2,Store C,29817,8587,0.287990
3,Store D,43945,16212,0.368916
4,Store E,44775,17879,0.399308
5,Store F,61019,18466,0.302627
6,Store G,42869,12741,0.297208


In [ ]:
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(dfq7['new_city'], dfq1['pph'], color ='blue',
        width = 0.4)
plt.xlabel("City",fontsize=15)
plt.ylabel("PPH (production per hour)",fontsize=15)
plt.title("PPH in each City",fontsize=20)
plt.show()